In [6]:
# Import packages
using JuMP
using HiGHS

# Creating the model
coffeemodel = Model(HiGHS.Optimizer)

# Declaring the vectors over types and blends
types = [:Columbian, :Brazilian, :Sumatran]
blends = [:Robust, :Light]

# Declare x indexed over types and blends
@variable(coffeemodel, x[types, blends] >= 0)

# Dict the available amount of beans
available_beans = Dict(:Columbian => 550, :Brazilian => 450, :Sumatran => 650)

# Dict the cost per pound of beans
bean_cost = Dict(:Columbian => 1.00, :Brazilian => 0.85, :Sumatran => 1.55)

# Dict the selling prices per pound for each blend
blend_prices = Dict(:Robust => 4.25, :Light => 3.95)

# Define constraints for each blend
# Robust Joe constraints
@constraint(coffeemodel, RSLowerBound, x[:Sumatran, :Robust] >= 0.60 * sum(x[i, :Robust] for i in types))
@constraint(coffeemodel, RSUpperBound, x[:Sumatran, :Robust] <= 0.75 * sum(x[i, :Robust] for i in types))
@constraint(coffeemodel, RSLowerColumbian, x[:Columbian, :Robust] >= 0.10 * sum(x[i, :Robust] for i in types))

# Light Joe constraints
@constraint(coffeemodel, LBLowerBrazilian, x[:Brazilian, :Light] >= 0.50 * sum(x[i, :Light] for i in types))
@constraint(coffeemodel, LBUpperBrazilian, x[:Brazilian, :Light] <= 0.60 * sum(x[i, :Light] for i in types))
@constraint(coffeemodel, LBUpperSumatran, x[:Sumatran, :Light] <= 0.20 * sum(x[i, :Light] for i in types))

# bean availability constraints
@constraint(coffeemodel, ColAvailable, sum(x[:Columbian, blend] for blend in blends) <= available_beans[:Columbian])
@constraint(coffeemodel, BraAvailable, sum(x[:Brazilian, blend] for blend in blends) <= available_beans[:Brazilian])
@constraint(coffeemodel, SumAvailable, sum(x[:Sumatran, blend] for blend in blends) <= available_beans[:Sumatran])

# finding the objective function
# Cost
cost = sum(bean_cost[i] * x[i, blend] for i in types for blend in blends)
# Revenue
revenue = sum(blend_prices[blend] * sum(x[i, blend] for i in types) for blend in blends)
# objective function (profit)
@objective(coffeemodel, Max, revenue - cost)

# Solve
optimize!(coffeemodel)

#Print values of all variables
for blend in blends
    println("$blend Blend:")
    for bean in types
        @show x[bean, blend]
    end
end
print(coffeemodel)




Running HiGHS 1.7.2 (git hash: 5ce7a2753): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e-01, 1e+00]
  Cost   [2e+00, 3e+00]
  Bound  [0e+00, 0e+00]
  RHS    [4e+02, 6e+02]
Presolving model
9 rows, 6 cols, 24 nonzeros  0s
9 rows, 6 cols, 24 nonzeros  0s
Presolve : Reductions: rows 9(-0); columns 6(-0); elements 24(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -4.4549962997e+01 Ph1: 8(10.175); Du: 6(44.55) 0s
          8     4.9025000000e+03 Pr: 0(0) 0s
Model   status      : Optimal
Simplex   iterations: 8
Objective value     :  4.9025000000e+03
HiGHS run time      :          0.00
Robust Blend:
x[bean, blend] = x[Columbian,Robust]
x[bean, blend] = x[Brazilian,Robust]
x[bean, blend] = x[Sumatran,Robust]
Light Blend:
x[bean, blend] = x[Columbian,Light]
x[bean, blend] = x[Brazilian,Light]
x[bean, blend] = x[Sumatran,Light]
Ma